In [ ]:
import os, sys
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.metrics import confusion_matrix

In [1]:
df = pd.read_csv('ParkinsonData/controlGroup.csv')
df.head()
df.info()

NameError: name 'pd' is not defined

In [ ]:
### Plotting number of healthy (0) and sick (1) persons
sns.set_style("white")
a = sns.countplot(df['status'],
              palette=["red", "blue"])
a.set(title = "Barplot of of healthy (0) and sick (1) persons recordings",
     xlabel = 'Health status',
     ylabel ='Number of Person')
plt.xlabel('Health status')
plt.ylabel('Number of Recordings')
###Print
h = df[df['status']==0]['status'].count()
s = df[df['status']==1]['status'].count()

print('Healthy :',h ,'Proportion:', round((h*100/(h+s)),2),"%")
print('Sick:',s,'Proportion:' ,round((s*100/(h+s)), 2),"%" )
plt.show();
plt.savefig("d1.png");

In [ ]:
##set plot parameters
fig, ax = plt.subplots(1,3,figsize=(15,9))
sns.boxplot(x='status',y='MDVP:Fo(Hz)', data=df, width=0.3,
                palette=["red", "blue"],
            ax=ax[0]).set(title = 'Average vocal fundamental frequency')
sns.boxplot(x='status',y='MDVP:Flo(Hz)', data=df, width=0.3,
                palette=["red", "blue"],
            ax=ax[1]).set(title = "Minimum vocal fundamental frequency")
sns.boxplot(x='status',y='MDVP:Fhi(Hz)', data=df, width=0.3,
                palette=["red", "blue"],
            ax=ax[2]).set(title = "Maximum vocal fundamental frequency")
plt.show();

In [ ]:
fig, axs = plt.subplots(ncols = 2, figsize = (20,9))
col_names = ["NHR","HNR"]
for i in range(0, len(col_names)):
    sns.boxplot(x='status', y=col_names[i], data=df,
                width=0.3, palette=["red", "blue"], ax = axs[i])
plt.show();

In [ ]:
correlation = round(df.corr(), 2)
mask = np.zeros_like(correlation)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(20, 14))
    ax = sns.heatmap(correlation, cmap="YlGnBu", annot = True, mask=mask, square=True, vmin=0, vmax=1).set(title = 'Correlation matrix')


In [ ]:
#Data preprocessing
df = df.dropna()
X = df.drop(['name', 'status'],axis=1)
y = df["status"]
print(X.shape)
print(y.shape)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#Data splitting
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.3, random_state = 150)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_train.shape)

In [ ]:
## RandoForestClassification Model
model = RandomForestClassifier(n_jobs=-1, random_state=50)
parameters={'max_depth':[1,2,3,4,5,10],
 'n_estimators':[1,2,3,4,5,10],
 'max_features':[1,2,3,4,5,10],
 'criterion':['gini'],
 'bootstrap':[True],
 'min_samples_leaf':[1,2,3,4]}

mod_forest = GridSearchCV(estimator=model,param_grid=parameters, cv=5)

%timeit

mod_forest.fit(X_train,y_train)
print(mod_forest.best_params_)

In [ ]:

##Test others models: Xgboost, KNN, Naive bayes, Decision Tree

ESTIMATORS = {"RandomForestClassifier": RandomForestClassifier(bootstrap=True, criterion='gini', max_depth=5, max_features=10, min_samples_leaf=2, n_estimators=10, random_state=50),
 "XgbClassifier": xgb.XGBClassifier(max_depth = 3, n_estimators = 10, objective = 'binary:logistic'),
 "K-nn": KNeighborsClassifier(n_neighbors=3),
 "Naive-bayes": GaussianNB(),
 "DecisionTreeClassifier": DecisionTreeClassifier(criterion='entropy', random_state=10, max_depth=6)
}

In [ ]:
def plot_conf_mat(conf_mat):
    """
    Plots a Confusion Matrix Using Seaborn heatmap.
    """
    fig, ax = plt.subplots(figsize=(3,3))
    ax = sns.heatmap(conf_mat,
                    annot=True, # Annotate the boxes with conf_mat info
                    cbar=False)
    plt.xlabel("True label")
    plt.ylabel("Predicted label")

In [ ]:
def draw_cf(y_test, y_preds):
    sns.set(font_scale=1.5)
    conf_mat = confusion_matrix(y_test, y_preds)
    plot_conf_mat(conf_mat);

In [ ]:
ESTIMATORS = {"RandomForestClassifier": RandomForestClassifier(bootstrap=True, criterion='gini',  max_depth=5, max_features=10, min_samples_leaf=2, n_estimators=10, random_state=50),
 "XgbClassifier": xgb.XGBClassifier(max_depth = 3, n_estimators = 10, objective = 'binary:logistic'),
 "K-nn": KNeighborsClassifier(n_neighbors=3),
 "Naive-bayes": GaussianNB(),
 "DecisionTreeClassifier": DecisionTreeClassifier(criterion='entropy', random_state=10, max_depth=6)
}

In [ ]:
ESTIMATORS["RandomForestClassifier"].fit(X_train, y_train)
y_test_predict = ESTIMATORS["RandomForestClassifier"].predict(X_test)
draw_cf(y_test, y_test_predict)

In [ ]:
ESTIMATORS["XgbClassifier"].fit(X_train, y_train)
y_test_predict = ESTIMATORS["XgbClassifier"].predict(X_test)
draw_cf(y_test, y_test_predict)

In [ ]:
ESTIMATORS["K-nn"].fit(X_train, y_train)
y_test_predict = ESTIMATORS["K-nn"].predict(X_test)
draw_cf(y_test, y_test_predict)

In [ ]:
ESTIMATORS["Naive-bayes"].fit(X_train, y_train)
y_test_predict = ESTIMATORS["Naive-bayes"].predict(X_test)
draw_cf(y_test, y_test_predict)

In [ ]:
ESTIMATORS["DecisionTreeClassifier"].fit(X_train, y_train)
y_test_predict = ESTIMATORS["DecisionTreeClassifier"].predict(X_test)
draw_cf(y_test, y_test_predict)

In [ ]:
##Get accuracy and f1_score

result = pd.DataFrame()
for name, estimator in ESTIMATORS.items():
    estimator.fit(X_train, y_train)
    y_test_predict = estimator.predict(X_test)
    accur = estimator.score(X_test, y_test)

    f1Scroe = f1_score(y_test,y_test_predict)
    result = result.append({'Model' : name,
                            'F1-Score (%)' : f1_score(y_test,y_test_predict)*100,
                            'Accuracy (%)' : accuracy_score(y_test, y_test_predict)*100,
                            'Recall (%)' : recall_score(y_test, y_test_predict)*100},
                            ignore_index = True)

In [ ]:
result.plot(kind="bar");